## Etapa previa: Correr archivos C

- Tener el compilador gcc instalado, se verifica utilizando el comando: gcc -v
- Para compilar una funcion de manera normal hacemos: gcc *nombre funcion*.c -o *nombre del archivo .o*
- Para el uso de ctypes utilizaremos : gcc -fPIC -shared -o *nombre funcion*.so *nombre funcion*.c
    - gcc es el compilador de c. Si no utilizamos g++ que es el de c++
    - shared : para crear una libreria compartida

## Nota, para correr en C++
- ctypes tambien sirve con un codigo en C++, vamos a tener que agregar la opcion extern "C" {< ADENTRO EL CODIGO>}
- Para compilar tambien cambia un poco: 
    - Ejemplo g++ -static -shared -o c_struct.so c_struct.cc 
    - El anterior ejemplo es lo que me sirvio, notar que se compila con g++ y no gcc



# Ctypes
Fuentes
<a href='https://www.youtube.com/watch?v=BcWtCV_3iqM&list=PLHwXkLexR9MCqqr5hD_8o5rbSXYHtfFiB&index=3'>Intro</a>
Ctypes es una libreria que nos va a permitir ejecutar codigo C/C++

Instalacion:
Basta con importarlo: import ctypes <br>
ctypes es una libreria de python <br>

## Pasos Preeliminares
Vamos a crear un archivo simple en C que imprime un string y lo vamos a llamar desde python

- Utilizamos el comando: gcc -fPIC -shared -o funcion.so funcion.c *De esta manera estamos compilando de forma compartida, s en so viene de shared*
- Incluimos la libreria de C creada utilizando ctypes.CDLL(**path_libreria**)
- El path de la libreria conviene escribirlo asi: C:\\Users\\Fede\\Desktop\\ingenieria_software\\miTeoria\\ctypes_practice\\test.so para no tener errores. Una manera aun mejor es utilizar el modulo os:
    - import os
    - path = os.getcwd() # get current working directory
    - ctypes.CDLL(os.path.join(path,*nombre archivo*.so))
- Generamos una funcion con esta libreria y le indicamos:
    - Lista con el tipo de los inputs
    - El tipo del output (solo puede ser uno)

Ejemplo:
func = clibrary.display3<br>
func.argtypes = [ctypes.c_char_p,ctypes.c_int]<br>
func.restype = ctypes.c_char_p<br>

print(func(b"Federico",32))<br>

## El problema de los tipos

En C los tipos son estaticos, es decir que una vez declarados solo reciben valores del mismo tipo. Esto no ocurre en python donde los tipos son dinamicos. Por otro lado, para el caso de los float hay distintas longitudes que le podemos asignar al numero, en cambio en python todo esto se ocupa en el tipo float.

Para solucionar este problema ctypes trae distintos tipos de datos para hacer la conexion entre tipos de C y tipos de python.

In [ ]:
import ctypes 

#algunos tipos
# ctypes.c_bool
# ctypes.c_buffer 



# Introduccion a los punteros

Primero empezamos por destacar que python no posee punteros y por lo tanto no posee una alocacion dinamica de memoria. 

En ctypes tenemos un tipo de dato dedicado a los punteros. Concretamente tnemos:

- ctypes.pointer
- ctypes.POINTER

En esta seccion lo que haremos sera:

- Crear dos funciones en C
- Respervar espacios de memoria (allocate memory) en C
- Retornar un puntero que apunta a espacio de memoria, en el codigo python
- Jugar con el puntero un poco
- Finalmente retornaremos el puntero devuelta al programa en C desde donde haremos un deallocate

Cabe destacar que python no puede hacer un *Allocate* de memoria, esto signficia que tampoco puede liberarla *Deallocate* por lo cual si traemos el puntero desde C sera necesario tambien liberar ese espacio. Esta liberacion del espacio la haremos en C.


## Metodo 1 pointer
Para crear un puntero podemos utilizar simplemente: pointer

- num = ctypes.c_int(100) #Creamos un objeto de ctypes
- ptr = ctypes.pointer(num)
- print(ptr.contents)

Observacion, no podemos hacer simplemente num = 100 y que ctypes nos de el valor del pointer. Tenes que declarar la variable con un tipo especifico de ctypes , de otra manera se obtiene un type error. Es decir, .pointer() espera una variable de tipo ctypes

## Metodo 2 POINTER
Este metodo permite crear un un puntero que en principio no apunta a nada en especifico, es decir, el mismo es es un puntero vacio. Sin embargo si fuera a llenarse debe ser con un entero.

Podemos hacer lo siguiente entonces:

- ptr2 = ctypes.POINTER(ctypes.c_int)
- ptr2.contents = num 
- print(ptr2.contents)

Cabe destacar que si queremos que nuestro puntero apunte a otro elemento por ejemplo:
- num2 = ctypes.c_int(200)
- ptr2.contents = num2 # y listo

### Notas

Claramente la diferencia entre pointer y POINTER es que POINTER crea punteros que pueden rellenarse a posteriori, mientras que pointer crea un puntero para una variable especifica.

La diferencia no es solamente esta, si no que tambien hay una diferencia de performance. POINTER es mucho mas rapido debido a que existe un sistema de cache interno en ctypes.

Por otro lado se hace la mencion de que al final del dia pointer utiliza a POINTER en su backend.

# Uso de Arreglos en ctypes

En esta parte voy a hacer distincion en dos tipos de maneras de trabajar con arrays

1. Arrays comunes de python
2. Arrays de numpy. En particular en esta parte vamos a aprovechar la libreria de numpy: <a href='https://numpy.org/doc/stable/reference/routines.ctypeslib.html'>np.ctypeslib</a>

Entonces vamos a ver dos maneras de trabajar con arreglos, 1. la estandar y 2. arreglos de numpy

## Metodologia de trabajo

En esta parte voy a sintetizar una manera de trabajar con ctypes:

1. Como declarar un arreglo en ctypes
2. Estructurar nuestra funcion en C
3. Como llamar a una funcion de C y entregarle nuestro arreglo
4. Como recibir un arreglo desde un modulo de C y llevarlo a un arreglo de Numpy




## 1. Como declarar un arreglo en ctypes

El esqueleto que vamos a seguir para no equivocarnos es el siguiente:

1. Crear un puntero dinamico en ctypes que apunte hace un arreglo de C
2. Generar nuestro arreglo 
3. Estructurar la funcion en C para que reciba correctamente el arreglo
4. Entregarselo a nuestra funcion de C de interes
5. Recibir un arreglo de C y llevarlo a Numpy

Advertencia sobre tipos:
Es conveniente tener en cuenta los tipos que asocia ctypes o ctypeslib con los tipos de C. Algunos son los siguientes:

- ctypes.c_float <-----> float (en C)
- numpy.float64 <-----> double (en C)

Aclaracion:
Siempr que yo trabaje con arreglos voy a prefereir hacerlos con numpy, los metodos estandar y con ctypeslib ambos se pueden aplicar a arreglos de numpy.

### Metodo estandar

- Creacion del puntero: 
<div style='color:yellow'>my_pointer = ctypes.POINTER(*tipo de ctypes*)</div>Aca *typo de ctypes* = ctypes.c_float / ctypes.c_double ; o cualquier otro

- Creacion de un arreglo:
<div style='color:yellow'>arr = np.numpy([1.1,1.2,etc])</div>

### Metodo con numpy.ctypeslib

- Creacion del puntero:
<div style='color:yellow'>my_pointer = numpy.ctypeslib.ndpointer(dtype=*tipo de numpy*,ndim=*Dimension del arreglo*, flags=*Flags*)</div>
- Creacion de un arreglo : La misma que el paso anterior

Observaciones:
Aparentemente las flags son las mismas que <a href='https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html'>numpy.ndarray.flags</a>


# 2. Estructurar nuestra funcion en C

Primero hay que determinar si la funcion recibe o no inputs. Por lo general es necesario el tamanio del arreglo.
- Sin outputs: <div style='color:skyblue'>void funcion(double *arr, int size)</div>
- Con outputs: <div style='color:skyblue'>double* funcion(*si la funcion tiene inputs especificarlos*){<br>
    float* arr = malloc(10*sizeof(int)); //Ver MUY IMPORTANTE<br>
    *asigne los valores del arreglo utilizando un for*<br>
}</div>

Observaciones: El asterisco * Indica que la variable que se recibe o entrega es un puntero, es decir hace referencia a una direccion de memoria.

Observaciones 2 - Con outputs: Aca hay un tema importantisimo que tiene que ver con la alocacion de memoria, o reserva de espacios de memoria. 

- Si vas a crear un arreglo en C, al crearlo vos estas reservando memoria y en ese espacio estas guardando algo, despues se requiere liberar ese espacio de manera explicita en C. Por lo cual todo proceso de creacion de un arreglo en C debe estar unido a un proceso de borrado de memoria en codigo C despues de utilizarla. Python no requiere esto, pues aparentemente hace esto automaticamente en el backend. Para borrar memoria en C utilizamos la funcion: free(*puntero aca*) . Lo conveniente es crear una funcion de liberacion de memoria en C, llamarla desde python y utilizarla para liberar el espacio.

- Por otro lado, la asignacion de memoria en C es dinamica, en python no, por lo cual si vos creas un objeto y luego le das otro valor este lo asigna en un nuevo espacio de memoria.

- MUY IMPORTANTE: Cuando creamos un arreglo dentro del scope de una funcion, la misma funciona como una cache que luego de que la funcion termina su proceso es borrado, por lo cual si vos queres retornar un elemento creado en una funcion tenes que llamar a la funcion de asignacion dinamica de memoria: malloc()

# Entregar el arreglo a la funcion de C 

En esta parte supondremos que nuestra funcion esta en la libreria clibrary compilada en el ejecutable clibrary.so de la manera mostrada al principio. El nombre de la funcion 

- Especificar los inputs de la funcion: <div style='color:yellow'>clibrary.funcion.argtypes = [my_pointer, ctypes.c_int]</div>
- Especificar los restornos de la funcion si los hay: <div style='color:yellow'>clibrary.funcion.restype = ctypes.double</div>
- Entregar los objetos: <div style='color:yellow'>clibrary.funcion(arr, len(np_arr))</div>




## Crear un arreglo en C

Un arreglo en C puede ser creado a partir de un puntero, por ejemplo:

- double *arr;

Se le puede dar sus elementos utilizando recursividad: (ciclo for)

- arr[i] = 1.780;

El arreglo en si solo se puede crear utilizando el puntero. Por otro lado es indispensable la sentencia malloc() que consiste en la asignacion dinamica de memoria en C, esto es debido a que de otra manera, no podemos sacarla del scope de la funcion.

- double *arr2 = malloc(50 *sizeof(double));
## Asignacion DINAMICA de memoria utilizando malloc()
Fuente: <a href='http://labsopa.dis.ulpgc.es/fso/cpp/intro_c/introc75.htm'>LINK</a>
Fuente 2: <a href='https://www.geeksforgeeks.org/dynamic-memory-allocation-in-c-using-malloc-calloc-free-and-realloc/'>LINK 2</a>

La funcion malloc() pertenece a la bilioteca < stdlib.h > por lo cual deberemos importarla en el modulo de C que vamayamos a utilizar.

De la manera mas simple malloc se utiliza de la siguiente forma:

ptr = (cast-type*) malloc(byte-size)

malloc es una funcion con las siguientes caracteristicas

- Pide como input el tamanio en bytes para la asignacion del tamanio de memoria. Se garantiza que esta sera memoria libre. 
- Retorna un puntero a la zona de memoria concedida. Este puntero es tipo void y se puede transformar en cualquier otra clase de puntero: void* ---> [ cualquier-puntero ]*

### La funcion de size of

Para poder conocer el tamanio en bytes que un determinado tipo de elemento tiene se utiliza la funcion sizeof(type)
Entonces si lo que queremos es conocer cuanto espacio en bytes se requiere reservar para N elementos del type = double, hacemos:

- byte-size = N*sizeof(double)

### Liberacion de memoria utilizando la funcion free()

Utilizando la funcion free se puede liberar la memoria:

- free(ptr)

### Resumen de uso:


1. Creacion del puntero ptr del typo cast-type que asigna una cantidad de memoria byte-size:
    - byte-size = N*sizeof(type) //N bytes reservados
    - ptr = (cast-type*) malloc(byte-size) //ptr es un puntero del tipo cast-type

2. Liberacion de la memoria una vez que ya no la vamos a utilizar mas
    - free(ptr)

## Recibir un arreglo de C y llevarlo a Numpy 

Para hacer esta conversion vamos a tener que utilizar nuevamente la libreria numpy.ctypeslib y en particular el metodo as_array():

- Create a numpy array from a ctypes array or POINTER
<div style='color:yellow'>numpy.ctypeslib.as_array(obj, shape=None)</div> 
- Para 1d shape=(1,)
- Ejemplo:<div style='color:yellow'> python_arr = np.ctypeslib.as_array(returned_array,shape=(10,))</div>
- Lo bueno de esto es que ya no hay que acceder a sus valores utilizando contents (returned_array.contents) y tener que ir imprimiendolos uno a uno

ctypes windwos:
https://www.dynamsoft.com/codepool/python-ctypes-load-call-shared-library.html 

ctypes package example:
https://github.com/joerick/python-ctypes-package-sample/blob/main/src/spam/summing.py